In [69]:
from psaw import PushshiftAPI
import praw
from prawcore import Forbidden, NotFound
from praw.exceptions import ClientException
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns

from scipy import stats

from pymongo import MongoClient, errors

from bson.json_util import loads, dumps
from bson.objectid import ObjectId

from datetime import datetime as dt

import boto3

import time

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_curve, auc, roc_auc_score
from sklearn.metrics import plot_confusion_matrix

from imblearn.ensemble import BalancedRandomForestClassifier

import pickle

from scipy import interp

import string
from pprint import pprint

In [2]:
creds = {}
with open ('/opt/cap1/.cap1', 'r') as fp:
    for line in fp:
        k, v = line.replace('\n','').split('\t')
        creds[k] = v

In [3]:
reddit = praw.Reddit(client_id=creds['REDDIT_ID'], 
    client_secret=creds['REDDIT_SECRET'],
    password=creds['REDDIT_PASSWORD'], 
    username=creds['REDDIT_USERNAME'],
    user_agent='accessAPI:v0.0.1 (by /u/{})'.format(creds['REDDIT_USERNAME']))

In [4]:
api = PushshiftAPI()

In [5]:
client = MongoClient('localhost', 27017)
db = client['cap2']

In [6]:
# get all comments into a list called 'docs', then make dataframe
query = {}
fields = { '_id': 0 }
# docs is about 1.3 GB in RAM
docs = list(db['comment'].find( query, fields ))

In [7]:
comms = pd.DataFrame(docs)

In [ ]:
# TODO add submission ids to the set of ids from trolls, probably append 't1_' to comments
# for ease of use

## assign labels, 'troll?', and 'child_of_troll?', and 'parent_of_troll?'

In [8]:
# create array of classes for target
classes = ['other', 'parent', 'child', 'troll']

In [9]:
# assign the label, 'troll?'' to each comment
troll_comment_ids_set = set(comms[~comms['author'].isna()]['id'])
comms['troll?'] = [int(mybool) for mybool in [
                    commid in troll_comment_ids_set for commid in comms['id']
                    ]]
np.sum(comms['troll?'])

6704

In [10]:
# is this comment in reply to a troll?
comms['child_of_troll?'] = [int(mybool) for mybool in [
                    pid.split('_')[1] in troll_comment_ids_set for pid in comms['parent_id']
                    ]]
np.sum(comms['child_of_troll?'])

2985

### only 57, praw never gave us children of troll comments...the only ones we have are from trolls replying to trolls

#### FIXED, used praw to get the troll comments, then used .refresh() to load the replies. worked ok

In [11]:
# did a troll reply to this?
troll_parent_ids_set = set(
                            [p.split('_')[1] for p in comms[~comms['author'].isna()]['parent_id']]
                        )
comms['parent_of_troll?'] = [int(myid in troll_parent_ids_set) for myid in comms['id']]

np.sum(comms['parent_of_troll?'])

1824

In [15]:
def printall(pd_obj):
    '''
    print every row and column in a pandas object
    '''
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(pd_obj)

In [16]:
for coll in db.list_collection_names():
    print(f'{coll:14}: {db[coll].count_documents({}):6}')

submission    :  14523
first_and_last:    964
comment       : 234008


In [17]:
comms

,author,author_created_utc,author_flair_css_class,author_flair_text,author_fullname,body,controversiality,created_utc,distinguished,gilded,...,author_flair_background_color,collapsed_because_crowd_control,mod_reports,mod_note,_fetched,author_cakeday,top_awarded_type,troll?,child_of_troll?,parent_of_troll?
0,1488reasons,1.452931e+09,nzfirst,,t2_tu7sn,yeah and he wants to consider it by limiting w...,0.0,1.467091e+09,None,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
1,1488reasons,1.452931e+09,nzfirst,,t2_tu7sn,We should just give up on security if it makes...,1.0,1.467089e+09,None,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
2,1488reasons,1.452931e+09,nzfirst,,t2_tu7sn,Mate if you think Aristotle would have anythin...,0.0,1.467089e+09,None,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
3,1488reasons,1.452931e+09,nzfirst,,t2_tu7sn,Oy now there's a nice quote out of context:\n\...,0.0,1.467088e+09,None,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
4,1488reasons,1.452931e+09,nzfirst,,t2_tu7sn,Living in your own filth isn't the kind of thi...,1.0,1.465718e+09,None,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234003,NaN,NaN,washington,WA,t2_95mme,Why can't you vot-oooh. I see the flair.,0.0,1.458231e+09,None,0.0,...,,NaN,[],NaN,True,NaN,NaN,0,1,0
234004,NaN,NaN,None,None,NaN,# B L Y A T,0.0,1.458048e+09,None,0.0,...,,NaN,[],NaN,True,NaN,NaN,0,1,0
234005,NaN,NaN,russia,RUS,t2_gz4o7,Starting when I wake up!,0.0,1.457985e+09,None,0.0,...,,NaN,[],NaN,True,NaN,NaN,0,1,0
234006,NaN,NaN,text-pcmr color-pcmr icon-steam,4690K@stock + gtx 970@1500 at 2560x1080,t2_jttbx,Sale thing in México,0.0,1.432011e+09,None,0.0,...,,NaN,[],NaN,True,NaN,NaN,0,1,0


In [21]:
comms['author_fullname']

0         t2_tu7sn
1         t2_tu7sn
2         t2_tu7sn
3         t2_tu7sn
4         t2_tu7sn
            ...   
234003    t2_95mme
234004         NaN
234005    t2_gz4o7
234006    t2_jttbx
234007    t2_e5a4u
Name: author_fullname, Length: 234008, dtype: object

In [23]:
api = PushshiftAPI()
result = api.redditor_subreddit_activity('t2_tu7sn')
result

{'comment': Counter(), 'submission': Counter()}

In [26]:
np.where(np.isnan(comms['author_fullname']), ['']*len(comms()), comms['author_fullname'])

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [39]:
# ignores the null author names
unique_author_fullname = np.unique(comms['author_fullname'].fillna(''))[1:]

In [58]:
with open('data/unique_author_fullname', 'w') as fp:
    fp.write('\n'.join(unique_author_fullname) + '\n')

In [54]:
len(unique_author_fullname)

145297

In [87]:
with open('data/unique_author_fullname', 'r') as fp:
    a2 = fp.read()

In [56]:
unique_author_fullname[0], unique_author_fullname[-1]

('t2_100267', 't2_zzyri')

In [50]:
btl = reddit.redditor(fullname="t2_z919g")
vars(btl)

{'_reddit': <praw.reddit.Reddit at 0x7fea00689af0>,
 '_fetched': False,
 '_listing_use_sort': True,
 '_fullname': 't2_z919g'}

In [52]:
btl._fetch()

In [53]:
vars(btl)

{'_reddit': <praw.reddit.Reddit at 0x7fea00689af0>,
 '_fetched': True,
 '_listing_use_sort': True,
 '_fullname': 't2_z919g',
 'name': 'BlackToLive',
 'is_employee': False,
 'icon_img': 'https://www.redditstatic.com/avatars/avatar_default_01_46D160.png',
 'pref_show_snoovatar': False,
 'is_friend': False,
 'created': 1467709998.0,
 'has_subscribed': True,
 'hide_from_robots': False,
 'created_utc': 1467681198.0,
 'link_karma': 1,
 'comment_karma': -7,
 'is_gold': False,
 'is_mod': False,
 'verified': True,
 'subreddit': {'default_set': True,
  'user_is_contributor': False,
  'banner_img': '',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '#46D160',
  'user_is_muted': False,
  'display_name': 'u_BlackToLive',
  'header_img': None,
  'title': '',
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://www.redditstatic.com/avatar

In [ ]:
# running this in detached terminal in ipython
# for author_fullname in df[0]: 
#     ...:     author = reddit.redditor(fullname=author_fullname) 
#     ...:     author._fetch 
        

In [59]:
db['author'].create_index('id', unique=True)
log = []
i = 0

In [75]:
for a in ['t2_108l58', 't2_10blbj']:
    i += 1
    print(f'searching for author with id {a}')
    author = reddit.redditor(fullname=a) 
    try:
        author._fetch()
    except (Forbidden, NotFound):
        print('Forbidden or NotFound')
        log.append(i)
        continue
    d = dict(vars(author))
    for key in ['_reddit']:
        _ = d.pop(key, None)
    try:
        db['author'].insert_one(d)
    except (errors.DuplicateKeyError, errors.InvalidDocument):
        continue

searching for author with id t2_108l58
Forbidden or NotFound
searching for author with id t2_10blbj
Forbidden or NotFound


In [71]:
list(db['author'].find({}))[-20:]

[{'_id': ObjectId('5ed954357bdff71f697b5215'),
  '_fetched': True,
  '_listing_use_sort': True,
  '_fullname': 't2_10b91w',
  'name': 'KingEimer',
  'is_employee': False,
  'icon_img': 'https://www.redditstatic.com/avatars/avatar_default_05_25B79F.png',
  'pref_show_snoovatar': False,
  'is_friend': False,
  'created': 1470854716.0,
  'has_subscribed': True,
  'hide_from_robots': False,
  'created_utc': 1470825916.0,
  'link_karma': 312,
  'comment_karma': 4698,
  'is_gold': False,
  'is_mod': False,
  'verified': True,
  'subreddit': {'default_set': True,
   'user_is_contributor': False,
   'banner_img': '',
   'restrict_posting': True,
   'user_is_banned': False,
   'free_form_reports': True,
   'community_icon': None,
   'show_media': True,
   'icon_color': '#25B79F',
   'user_is_muted': False,
   'display_name': 'u_KingEimer',
   'header_img': None,
   'title': '',
   'previous_names': [],
   'over_18': False,
   'icon_size': [256, 256],
   'primary_color': '',
   'icon_img': 'http

In [86]:
for coll in db.list_collection_names():
    print(f'{coll:14}: {db[coll].count_documents({}):6}')

submission    :  14523
author        :   1029
first_and_last:    964
comment       : 234008


In [64]:
145297/443

327.9841986455982

In [65]:
15*327/60/24

3.40625

In [77]:
printall(comms[comms['author_fullname']=='t2_108l58'])

       author  author_created_utc author_flair_css_class author_flair_text  \
179924    NaN                 NaN                   None              None   

       author_fullname                body  controversiality   created_utc  \
179924       t2_108l58  Oh shit. Thanks OP               0.0  1.494683e+09   

       distinguished  gilded       id    link_id  nest_level  parent_id  \
179924          None     0.0  dhi85sc  t3_6aqab6         NaN  t3_6aqab6   

        reply_delay  retrieved_on  score stickied subreddit subreddit_id  \
179924          NaN           NaN      1    False       NaN     t5_2sgp1   

             created edited score_hidden can_gild is_submitter no_follow  \
179924  1.494711e+09  False        False     True        False      True   

                                                permalink send_replies  \
179924  /r/pcmasterrace/comments/6aqab6/psa_utorrent_a...         True   

       subreddit_type gildings  updated_utc author_flair_template_id  \
179924  

In [79]:
d = {
            "author": "HumourousOutline",
            "body": "Oh shit. Thanks OP",
            "can_gild": True,
            "created_utc": 1494682651,
            "edited": False,
            "id": "dhi85sc",
            "link_id": "t3_6aqab6",
            "parent_id": "t3_6aqab6",
            "retrieved_on": 1494682652,
            "stickied": False,
            "subreddit": "pcmasterrace",
            "subreddit_id": "t5_2sgp1"
        }

NameError: name 'true' is not defined

In [80]:
comms["can_gild"]

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
          ...  
234003    False
234004    False
234005    False
234006     True
234007     True
Name: can_gild, Length: 234008, dtype: object

In [83]:
gen = api.search_comments(id='dhi85sc')
d = next(gen).d_

In [84]:
d

{'all_awardings': [],
 'approved_at_utc': None,
 'associated_award': None,
 'author': 'Dr-Cupid',
 'author_flair_background_color': '',
 'author_flair_css_class': '',
 'author_flair_richtext': [{'e': 'text', 't': '16'}],
 'author_flair_template_id': None,
 'author_flair_text': '16',
 'author_flair_text_color': 'dark',
 'author_flair_type': 'richtext',
 'author_fullname': 't2_2victs0v',
 'author_patreon_flair': False,
 'author_premium': True,
 'awarders': [],
 'banned_at_utc': None,
 'body': 'I’m still confused',
 'can_mod_post': False,
 'collapsed': False,
 'collapsed_because_crowd_control': None,
 'collapsed_reason': None,
 'created_utc': 1591297687,
 'distinguished': None,
 'edited': False,
 'gildings': {},
 'id': 'fsw2r9g',
 'is_submitter': True,
 'link_id': 't3_gtuo3n',
 'locked': False,
 'no_follow': True,
 'parent_id': 't1_fsfh9p3',
 'permalink': '/r/teenagers/comments/gtuo3n/we_as_the_united_states_of_america_have_to_do/fsw2r9g/',
 'retrieved_on': 1591302481,
 'score': 1,
 'send

In [85]:
# In [71]: i, log  
# Out[71]: (1011, [673, 718, 852])
unique_author_fullname[[673, 718, 852]]

array(['t2_10cq4g', 't2_10deft', 't2_10gkgq'], dtype=object)